# Hash Tables

A **hash table** is a data structure that implements symbol tables  with constant-time performance for core operations, provided that search keys are standard data types or simply defined. It uses different access to the data compared to other BSTs, and doesn't support ordered operations.

## Hashing and Hash Functions

The basic idea of **hashing** is to save items in a *key-indexed table* where the index is a function of the key. A **hash function** is a method for computing the array index from the given key.

Issues:

- Computing the hash function
- Equality test: method for checking whether two keys are equal
- Collision resolution: algorithm and data structure to handle two keys that hash to the same array index

Hashing is a classic space-time tradeoff. With no space limitation, you use a trivial hash function with the key as the index. With no time limitation, you hash everything to the same place (use a trivial collision resolution) and do sequential search. In the real world with space and time limitations, you use hashing.

**Computing the hash function**

The idealistic goal is to scramble the keys uniformly to produce a table index - it's efficiently computable and each table index is equally likely for each key. You have an array the size of $M$ and $N$ keys to insert.

There are usually built-in methods in your language of choice for doing this with different data types.

**Modular hashing:**

- Hash code: an integer between $-2^{31}$ and $2^{31} - 1$
- Hash function: you want it to produce an integer between $0$ and $M - 1$ (for use as the array index). $M$ is typically a prime or a power of 2

The math that makes this work is to hash the key, get rid of the sign by taking `and` with $2^{31}$ in hexidecimal, then mod with $M$: `(hash(key) & 0x7fffffff) % M`.

The **uniform hashing assumption** is that each key is equally likely to hash to an integer between $0$ and $M - 1$. Think of throwing balls at random into $M$ bins. The **birthday problem** (when will you expect your first collision, or when 2 people in a room have the same birthday) is you'd expect two balls in the same bin after ~$\sqrt{\pi M / 2}$ tosses. The **coupon collector** assumption is that you expect every bin has $\geq 1$ ball after ~$M \ln M$ tosses. And the **load balancing** problem says that after $M$ tosses, you expect that the most loaded bin has $\Theta (\log M / \log \log M)$ balls.

## Separate Chaining

**Separate chaining** is a collision resolution strategy that makes use of the linked list data structure. A collision happens when two distinct keys hash to the same index.

The **birthday problem** said that you need huge amounts of memory (quadratic) to avoid a collision, which is too much in practice. And the **coupon collector** and **load balancing** problems said that collisions will be evenly distributed. 

So, how do you handle collisions efficiently?

Separate chaining is one solution that uses an array of $M \lt N$ linked lists:

- Hash: map kay to integer $i$ between $0$ and $M - 1$
- Insert: put at front of $i^{th}$ chain (if not already there)
- Search: need to search only $i^{th}$ chain

Proposition is that under the uniform hashing assumption, the probability that the number of keys in a list is within a constant factor of $N/M$ is extremely close to $1$. The proof shows that the distribution of the list size obeys a binomial distribution.

The consequence is that the number of probes for search/insert is proportional to $N/M$:

- $M$ is too large $\implies$ too many empty chains
- $M$ is too small $\implies$ chains are too long
- Typical choice: $M \text{~} N/5$ for constant-time operations

## Linear Probing

**Linear probing** is another popular collision resolution method. It uses **open addressing**, so when a key collides, it finds the next empty slot and puts the data there. Instead of keeping linked lists in positions along an array, you just allocate an array the size of the data.

The idea is to first hash the key to map it to an integer $i$ between $0$ and $M-1$. Then insert the key at index $i$ in the array if that slot if free, if not, try $i+1$, $i+2$, etc. To search, get the hash of the key, check index $i$ in the array, if that slot is occupied but there's no match, try $i+1$, $i+2$, etc. until you find it or hit an empty slot. A group of keys in consecutive slots in the array are called a cluster.

The size of the array $M$ *must be* greater than the number of key-value pairs $N$. Implementations usually include array re-sizing code to adjust $M$ for the size of the data.

In the earlier days of computing where memory was at a premium, a lot of care went into preventing empty keys or linked lists. One observation was that new keys were likely to hash into the middle of big clusters, and big clusters were likely to get longer.

**Knuth's parking problem** frames it as cars parking on a one-way street, if every car starts looking for a place at a random time, how far do they have to go to find a spot? He could show that if half the spots are occupied ($M/2$), on average, half find a spot in one place, half go one more spot (mean displacement of ~$3/2$). But when it's full ($M$ cars), the mean displacement is ~$\sqrt{\pi M/8}$.

**Proposition:** under uniform hashing assumption, the average number of probes in a linear probing hash table of size $M$ that contains $N = \alpha M$ keys is:

**Search hit:**
$$
\text{~} \frac{1}{2} \bigg( 1 + \frac{1}{(1-\alpha)} \bigg)
$$

**Search miss or Insert:**
$$
\text{~} \frac{1}{2} \bigg( 1 + \frac{1}{(1-\alpha)^2} \bigg)
$$

**Parameters:**

- $M$ too large $\implies$ too many empty array entries
- $M$ too small $\implies$ search time blows up
- Typical choice: $\alpha = N/M \text{~} \frac{1}{2}$ so the number of probes for a hit is ~3/2 and for search miss/insert is ~5/2

In [1]:
class LinearProbingHashST:
    def __init__(self, M):
        self.M = M
        self.key_arr = [None] * M
        self.vals_arr = [None] * M
        self.count = 0
    
    def hash_function(self, key):
        return (hash(key) & 0x7fffffff) % self.M

    def put(self, key, value):
        # Inserts key and value into arrays at hashed or next avail index
        i = self.hash_function(key)
        while self.key_arr[i] is not None and self.key_arr[i] != key:
            # Will be inf loop if array is full and doesn't contain key
            i = (i + 1) % self.M
        self.key_arr[i] = key
        self.vals_arr[i] = value
        self.count += 1
    
    def get(self, key):
        i = self.hash_function(key)
        while self.key_arr[i] is not None:
            # Will be inf loop if array is full and doesn't contain key
            if key == self.key_arr[i]:
                return self.vals_arr[i]
            i = (i + 1) % self.M
        return None
    
    def __contains__(self, key):
        return self.get(key) is not None
    
    def __str__(self):
        return str(self.key_arr)
    
    def __len__(self):
        return self.count

In [2]:
lin_probe = LinearProbingHashST(11)
for letter in ['E', 'S', 'T', 'Q', 'A', 'B', 'Z', 'H', 'P']:
    lin_probe.put(letter, lin_probe.hash_function(letter))
print(lin_probe)
for letter in ['E', 'S', 'T', 'Q', 'A', 'B', 'Z', 'H', 'P']:
    print(lin_probe.get(letter))

['P', None, None, 'T', 'A', 'Z', 'H', 'E', 'S', 'Q', 'B']
7
7
3
8
4
9
3
6
4


## Hash Table Context

To come

## Search Tree Summary

The worst case (WC) is after $N$ inserts, and the average case (AC) is after $N$ random inserts.

The height of any red-black BST on $n$ keys (regardless of the order of insertion) is guaranteed to be between $\log⁡_{2} n$ and $2 \log_{⁡2}n$.

| Implementation | WC Search | WC Insert | WC Delete | AC Search | AC Insert | AC Delete | Ordered Iteration? |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Sequential Search (unordered list) | $N$ | $N$ | $N$ | $N/2$ | $N$ | $N/2$ | No |
| Binary Search (ordered array) | $\lg N$ | $N$ | $N$ | $\lg N$ | $N/2$ | $N/2$ | Yes |
| Binary Search Tree (BST) | $N$ | $N$ | $N$ | $1.39 \lg N$ | $1.39 \lg N$ | ? | Yes |
| 2-3 Tree | $c \lg N$ | $c \lg N$ | $c \lg N$ | $c \lg N$ | $c \lg N$ | $c \lg N$ | Yes |
| Red-Black BST | $2 \lg N$ | $2 \lg N$ | $2 \lg N$ | $1.00 \lg N$\* | $1.00 \lg N$\* | $1.00 \lg N$ | Yes |
| Separate Chaining | $\lg N$\*\* | $\lg N$\*\* | $\lg N$\*\* | $3 \cdot 5$\*\* | $3 \cdot 5$\*\* | $3 \cdot 5$\*\* | No |
| Linear Probing | $\lg N$\*\* | $\lg N$\*\* | $\lg N$\*\* | $3 \cdot 5$\*\* | $3 \cdot 5$\*\* | $3 \cdot 5$\*\* | No |

\* Exact coefficient unknown but extremely close to 1  
\*\* Under uniform hashing assumption